# Aim

For each article we extracted all keyword occurrences and then identified the sentences containing keyword occurrences and generating separate keyword-sentence pairs ("opinion contexts").

This dataset of keyword-sentences pairs was then manually coded for sentiment by reserachers.

During the manual coding, the dataset was further cleaned and refinsed as, for instance, duplicates of articles was still found and removed. This is because LexisNexus captured updates to the same article as separate entries.

The final dataset used for analysis is the dataset based on the keyword-sentence pairs (full text was also a column in that data) after manual coding and inspection.

# Set up and get data

In [ ]:
import os
import pickle

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import psutil

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()

In [ ]:
pd.set_option('display.max_colwidth', None)

### Constant

In [ ]:
DIR_DATA = os.environ.get("DIR_DATA_INTERIM")

In [ ]:
DIR_DATA_EXTRA = os.environ.get("DIR_DATA_EXTRA")

In [ ]:
# prominence
term_freqs_nm = "kword_rawfreq_2W-MON.csv"
doc_freqs_nm = "kword_docfreq_2W-MON.csv"
metrics_nm = "kword_rfrdf_2W-MON.csv"

In [ ]:
# sentiment
sentiment_nm = "preproc_kword_sent.pickle"

In [ ]:
# keywords to be excluded because of low frequency in the corpus
EXCLUDE_KWORDS = ['behav_insight', 'behavioural_economist', 'behav_analysis', 
                  'chater', 'american_behav_scientists', 'irrational_econ', 'nudge_choice']

In [ ]:
EXCLUDE_NONKWORDS = ["herd_immunity", "behavioural_fatigue"]

## Import data

In [ ]:
doc_freqs = pd.read_csv(os.path.join(DIR_DATA, doc_freqs_nm))

In [ ]:
term_freqs = pd.read_csv(os.path.join(DIR_DATA, term_freqs_nm))

In [ ]:
metrics = pd.read_csv(os.path.join(DIR_DATA, metrics_nm))

In [ ]:
with open(os.path.join(DIR_DATA, sentiment_nm), "rb") as input_file:
        sentiments = pickle.load(input_file)

In [ ]:
# save as csv for visual ispection by others
sentiments.to_csv(os.path.join(DIR_DATA, "preprc_kword_sent.csv"))


### Number of keywords per opinion context

Remove non keywords: "herd_immunity" and "behavioural_fatigue"

In [ ]:
sentiments[sentiments.kword.isin(EXCLUDE_NONKWORDS)].shape

In [ ]:
sentiments[~sentiments.kword.isin(EXCLUDE_NONKWORDS)].shape

In [ ]:
nonkeyword_cases = sentiments[sentiments.kword.isin(EXCLUDE_NONKWORDS)]

In [ ]:
sentiments = sentiments[~sentiments.kword.isin(EXCLUDE_NONKWORDS)].copy()

In [ ]:
sentiments[['article_id', 'opinion_context_id']].drop_duplicates().nunique()

Before removing keywords based on low frequency

In [ ]:
sentiments.columns

In [ ]:
sentiments = sentiments[['article_id', 'title', 'pub_date', 'subkword', 'opinion_context', 'opinion_context_id',
       'keyword_sentiment', 'refers_to_gov', 'gov_sentiment', 'kword', 'pub_date_dt']].copy()

### After removing opinion context that contained "behavioural fatigue" or "herd immunity"
641 articles were left
This means that 6 articles only included "behavioural fatigue" or "herd immunity" as keywords

In [ ]:
sentiments.article_id.nunique()

In [ ]:
sentiments.shape

It means 6 articles only contained "behavioural fatigue" or "herd immunity". Which ones?

In [ ]:
nonkeyword_cases[~nonkeyword_cases.title.isin(sentiments.title)].title.nunique()

In [ ]:
nonkeyword_cases[~nonkeyword_cases.title.isin(sentiments.title)].title.unique()

#### Number of sentences-keyword pairs

In [ ]:
sentiments.opinion_context_id.nunique()

Doesn't match number of rows, why?

In [ ]:
sentiments.sort_values('opinion_context_id')

Some `opinion_context_id` are NA values.

In [ ]:
sentiments[np.isnan(sentiments['opinion_context_id'])].shape

In [ ]:
# duplicate opinion_context_id
sentiments[sentiments.duplicated('opinion_context_id')].opinion_context_id.tolist()


In [ ]:
len(sentiments[sentiments.duplicated('opinion_context_id')].opinion_context_id.tolist())

In [ ]:
1457+146

In [ ]:
# duplicate cases of opinion_context_id
sentiments[sentiments.opinion_context_id.isin(
    sentiments[sentiments.duplicated('opinion_context_id')].opinion_context_id.tolist())].sort_values(
    'opinion_context_id')

May be due to when the two batches were merged (each with opinion context id starting from 1)

### Give unique id for each sentence-subkeyword pair

In [ ]:
import random

In [ ]:
sentiments['subkw_sentence_pair_id'] = np.nan

In [ ]:
sentiments['subkw_sentence_pair_id'] = sentiments.groupby(['opinion_context', 'subkword'])['subkw_sentence_pair_id'].transform(lambda x: random.random())

In [ ]:
sentiments['subkw_sentence_pair_id'].nunique()

In [ ]:
sentiments[sentiments.subkw_sentence_pair_id.isin(
    sentiments[sentiments.duplicated('subkw_sentence_pair_id')].subkw_sentence_pair_id)].to_csv("../../sandpit/news_still_duplicates.csv")

In [ ]:
os.getcwd()

#### need to consider article_id as well

As some opinion context are repeated

In [ ]:
sentiments['subkw_sentence_pair_id'] = sentiments.groupby(
    ['article_id', 'opinion_context', 'subkword'])['subkw_sentence_pair_id'].transform(lambda x: random.random())

In [ ]:
sentiments['subkw_sentence_pair_id'].nunique()

## ok

### Number of unique sentences

In [ ]:
sentiments.opinion_context.nunique()

In [ ]:
sorted(sentiments.opinion_context)

In [ ]:
sentiments[['subkword', 'kword', 'opinion_context_id', 'opinion_context']].sort_values('opinion_context')

In [ ]:
pd.DataFrame(sentiments.groupby("opinion_context").subkword.value_counts())

#### Apply an id to each sentence

We need to consider sentence - article for unique ids

In [ ]:
import random

In [ ]:
sentiments.opinion_context.nunique()

In [ ]:
sentiments['sentence_id'] = np.nan

In [ ]:
sentiments['sentence_id'] = sentiments.groupby(
    ['article_id', 'title', 'opinion_context', 'pub_date_dt'])['sentence_id'].transform(lambda x: random.random())

In [ ]:
sentiments['sentence_id'].nunique()

In [ ]:
from src.news_media.get_keywords_trend import *

In [ ]:
uk_news = NewsArticles()

In [ ]:
opinions_data = uk_news.data_raw.drop('full_text', axis=1).copy()

In [ ]:
opinions_data = opinions_data[~opinions_data['kword'].isin(EXCLUDE_NONKWORDS)].copy()

In [ ]:
opinions_data[['article_id', 'title', 'opinion_context', 'pub_date_dt']].drop_duplicates()

In [ ]:
opinions_data.shape

In [ ]:
opinions_data[['article_id', 'title', 'opinion_context', 'pub_date_dt']].drop_duplicates()

In [ ]:
# number of subkeywords per sentence
sentiments.groupby("sentence_id").subkword.count().values

In [ ]:
len(sentiments.groupby("sentence_id").subkword.count().values)

In [ ]:
min(sentiments.groupby("sentence_id").subkword.count().values)

In [ ]:
max(sentiments.groupby("sentence_id").subkword.count().values)

In [ ]:
np.median(sentiments.groupby("sentence_id").subkword.count().values)

In [ ]:
# historgram
fig, ax = plt.subplots(figsize=(10,10))  

plt.hist(sentiments.groupby("sentence_id").subkword.count().values, alpha=0.5)

plt.show()

# Frequency of kwords and subkeywords

In [ ]:
sentiments.subkword.unique()

In [ ]:
# named actors
sentiments[sentiments.subkword.isin(
    ['chater', 'michie', 'halpern', 'sunstein', 'thaler', 'kahneman'])].groupby('subkword').count()

In [ ]:
term_freqs.iloc[:, 2:].sum(axis=0).sort_values(ascending=False)

In [ ]:
sentiments.groupby('kword').sentence_id.count().sort_values(ascending=False)

In [ ]:
term_freqs.drop(['herd_immunity', 'behavioural_fatigue', 'fortnight_starting', 'word_count'], axis=1).sum().sum()